In [ ]:
import torch
from ultralytics import YOLO
import os
import shutil # Для перемещения файла

# --- НАСТРОЙКИ ---
# Укажите путь к вашему файлу best.pt
# Обычно он сохраняется в директории runs/train/expN/weights/best.pt или,
# согласно вашему скрипту обучения: /kaggle/working/yolo_weights/motor_detector/weights/best.pt
path_to_best_pt = "epoch20.pt"

# Укажите желаемый путь для сохранения файла .torchscript
# Ваш скрипт ожидает что-то вроде '/kaggle/input/fgdsgsdg/.torchscript'
# Сначала мы сохраним его в /kaggle/working/, а затем вы сможете создать из него датасет Kaggle.
# Или вы можете назвать его '.torchscript', если ваш скрипт ожидает именно такое имя файла.
output_torchscript_filename = "best_model.torchscript" # Имя выходного файла
output_torchscript_dir = "/content/"
output_torchscript_path = os.path.join(output_torchscript_dir, output_torchscript_filename)

# Размер изображения (imgsz), использованный при обучении.
# В вашем скрипте обучения указано imgsz=1080.
img_size = 960
# --- КОНЕЦ НАСТРОЕК ---

def convert_pt_to_torchscript(pt_path, output_path, imgsz_export):
    """
    Преобразует файл .pt в формат TorchScript.

    Args:
        pt_path (str): Путь к файлу .pt.
        output_path (str): Путь для сохранения модели TorchScript.
        imgsz_export (int or tuple): Размер изображения для экспорта (должен соответствовать обучению).
    """
    if not os.path.exists(pt_path):
        print(f"Ошибка: Файл .pt не найден по пути: {pt_path}")
        return

    print(f"Загрузка модели из файла .pt: {pt_path}")
    try:
        model = YOLO(pt_path)  # Загрузка вашей обученной модели
    except Exception as e:
        print(f"Ошибка при загрузке модели из {pt_path}: {e}")
        return

    print(f"Экспорт модели в формат TorchScript (imgsz={imgsz_export})...")
    try:
        # Используем метод export() из Ultralytics
        # format='torchscript'
        # imgsz должен соответствовать размеру, на котором модель была обучена и будет использоваться для инференса
        # half=True, так как ваш скрипт инференса использует .half()
        # optimize=False часто более стабильно, но можно попробовать True для потенциального ускорения
        exported_file_path = model.export(format='torchscript', imgsz=imgsz_export, half=True, optimize=False)

        # Метод export() обычно сохраняет файл в той же директории, что и исходный .pt файл,
        # с тем же именем, но с расширением .torchscript.
        # Переименуем и/или переместим его в желаемое место.

        # Создаем выходную директорию, если она не существует
        os.makedirs(os.path.dirname(output_path), exist_ok=True)

        shutil.move(exported_file_path, output_path)
        print(f"Модель успешно экспортирована в TorchScript и сохранена по пути: {output_path}")

    except Exception as e:
        print(f"Ошибка во время экспорта в TorchScript с использованием model.export(): {e}")
        print("Возможные причины:")
        print("- Убедитесь, что у вас установлена последняя версия Ultralytics: pip install -U ultralytics")
        print("- Параметр 'imgsz' может быть неверным. Убедитесь, что он соответствует настройкам обучения.")
        print("Попытка экспорта с использованием torch.jit.trace в качестве запасного варианта...")
        try:
            # Получаем саму PyTorch модель
            pytorch_model = model.model.eval()
            device = next(pytorch_model.parameters()).device
            dummy_input = torch.randn(1, 3, imgsz_export, imgsz_export).to(device)
            if True: # соответствует half=True
                dummy_input = dummy_input.half()

            traced_script_module = torch.jit.trace(pytorch_model, dummy_input)
            traced_script_module.save(output_path)
            print(f"Модель успешно экспортирована в TorchScript с помощью torch.jit.trace и сохранена по пути: {output_path}")
        except Exception as e_trace:
            print(f"Ошибка во время экспорта с использованием torch.jit.trace: {e_trace}")


if __name__ == '__main__':
    # Убедитесь, что path_to_best_pt указывает на ваш фактический файл best.pt
    # Если вы только что запустили скрипт обучения из вашего вопроса, путь должен быть правильным.
    if not os.path.exists(path_to_best_pt):
        print(f"ПРЕДУПРЕЖДЕНИЕ: Файл best.pt не найден по указанному пути: {path_to_best_pt}")
        print("Пожалуйста, проверьте и исправьте переменную 'path_to_best_pt' в этом скрипте.")
    else:
        convert_pt_to_torchscript(path_to_best_pt, output_torchscript_path, img_size)

    print("\nПроцесс конвертации завершен.")
    if os.path.exists(output_torchscript_path):
        print(f"Выходной файл TorchScript: {output_torchscript_path}")
        print("\nДальнейшие шаги в Kaggle:")
        print(f"1. Ваш скрипт для отправки результатов ожидает модель по пути: '{'/kaggle/input/fgdsgsdg/.torchscript'}'")
        print(f"2. Чтобы использовать созданный файл '{output_torchscript_path}', вам нужно:")
        print(f"   a. Нажать 'Save Version' в этом ноутбуке (где вы запускаете этот скрипт конвертации).")
        print(f"   b. После завершения сохранения, перейти в раздел 'Output' этой версии ноутбука.")
        print(f"   c. Найти файл '{output_torchscript_filename}' и нажать '...' -> 'New Dataset'.")
        print(f"   d. Дать новому датасету имя (например, 'my-converted-motor-model').")
        print(f"   e. В вашем ноутбуке для отправки результатов, добавить этот новый датасет ('+ Add data').")
        print(f"   f. Обновить переменную 'model_path' в скрипте отправки, чтобы она указывала на файл в новом датасете, например:")
        print(f"      `model_path = '/kaggle/input/my-converted-motor-model/{output_torchscript_filename}'`")
        print(f"      Или, если вы назвали датасет 'fgdsgsdg' и файл внутри него '.torchscript', то оригинальный путь будет работать.")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Загрузка модели из файла .pt: epoch20.pt
Экспорт модели в формат TorchScript (imgsz=960)...
Ultralytics 8.3.134 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel CPUs. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLO11m summary (fused): 125 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs

PyTorch: starting from 'epoch20.pt' with input shape (1, 3, 960, 960) BCHW and output shape(s) (1, 5, 18900) (115.5 MB)

TorchScript: starting export with torch 2.6.0+cu124...
TorchScript: export success ✅ 17.5s, saved as 'epoch20.torchscript' (77.2 MB)

Export complete (29.2s)
Results saved 

In [ ]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling